In [ ]:
import numpy as np
import pandas as pd

from motse import MoTSE

%load_ext autoreload
%autoreload 2

# Configuration

In [ ]:
scratch_result_path = "../results/QM9/GCN/1000/"
transfer_results_path = "../results/QM9/GCN/10000->1000/"
similarity_path = "../results/QM9/GCN/10000/"
probe_data = "zinc500"
target_tasks = ["mu","alpha","homo","lumo","gap","r2","zpve","u0","u298","h298","g298","cv"]
n_recoms = 3

# Evaluation

In [ ]:
scratch_list, moste_list, best_list = [],[],[]
for target_task in target_tasks:
    # data loading
    scratch_result = pd.read_csv(f"{scratch_result_path}results.csv",index_col=0).loc[target_task].values[0]
    transfer_results = pd.read_csv(f"{transfer_results_path}{target_task}.csv", header=0)
    similarity = pd.read_csv(f"{similarity_path}{probe_data}/{target_task}.csv", header=0)
    
    source_tasks = transfer_results['source task'].values.tolist()
    transfer_results = transfer_results['r2'].values.tolist()
    similarity = similarity['motse'].values.tolist()

    drop_id = source_tasks.index(target_task)
    source_tasks.pop(drop_id)
    transfer_results.pop(drop_id)
    similarity.pop(drop_id)
    
    # evaluating
    motse = MoTSE()  
    scratch, motse, best = motse.eval_source_task_recom(n_recoms, target_task, source_tasks, scratch_result,np.array(transfer_results),np.array(similarity))
    scratch_list.append(scratch)
    moste_list.append(motse)
    best_list.append(best)
print(f"[mean] scratch:{np.mean(scratch_list):.3f}, motse:{np.mean(moste_list):.3f}, best:{np.mean(best_list)}")